chicken dinner github : https://github.com/crflynn/chicken-dinner
<br>pubg api-assets : https://github.com/pubg/api-assets

### Aggregate statistics

import library

In [1]:
## for calling dataset
import requests

## parsing dataset
from chicken_dinner.pubgapi import PUBG
from chicken_dinner.constants import COLORS
from chicken_dinner.constants import map_dimensions

import json
import numpy as np
import pandas as pd
import os

from tqdm import tqdm
from datetime import datetime
import config

In [2]:
api_key = config.api_key
match_sample_url = config.match_sample_url
headers = config.headers

requesting PUBG API data

In [3]:
response = requests.get(match_sample_url, headers=headers)
response

<Response [200]>

In [4]:
r_dict = json.loads(response.text)

In [5]:
file_date = r_dict['data']['attributes']['createdAt'][:r_dict['data']['attributes']['createdAt'].find('T')]
file_date

'2024-02-17'

In [6]:
os.mkdir(f'data/{file_date}')

In [7]:
## save data into json file
sample_file_path = f'data/{file_date}/{file_date}_smp_data.json'
sample_file_path

'data/2024-02-17/2024-02-17_smp_data.json'

In [8]:
with open(sample_file_path, 'w') as f:
    json.dump(r_dict, f)

reloading the previously saved sample data

In [9]:
with open(sample_file_path, 'r') as f:
    sample_data = json.load(f)

In [10]:
sample_data

{'data': {'type': 'sample',
  'id': '6f0faa6b-bb3b-4438-bfca-c171e634cc21',
  'attributes': {'createdAt': '2024-02-17T00:00:00Z',
   'titleId': 'bluehole-pubg',
   'shardId': 'kakao'},
  'relationships': {'matches': {'data': [{'type': 'match',
      'id': '762369d9-fd81-4b5a-9bd7-3aca6428a64b'},
     {'type': 'match', 'id': '1c7336ad-ab2e-43f5-8a15-4fdfffbee81c'},
     {'type': 'match', 'id': '023492b1-60b2-4c10-9a9c-3dde01168daa'},
     {'type': 'match', 'id': '4b997684-47d7-4c9a-9173-dc3fd26d75fd'},
     {'type': 'match', 'id': '2e6da0e6-effa-4331-864f-8ed4b8bad683'},
     {'type': 'match', 'id': 'f78652f6-567c-4870-96d4-0e0cfdf4a6aa'},
     {'type': 'match', 'id': '09866426-304b-44b4-92d7-f476892377f5'},
     {'type': 'match', 'id': '489b6916-3fba-4f99-bb96-ee1eb854f3b2'},
     {'type': 'match', 'id': 'f2663b54-c5d5-4432-b0cd-2052dc87d36d'},
     {'type': 'match', 'id': '601ee133-d236-47fe-b7d7-1068ae14453f'},
     {'type': 'match', 'id': '696960da-72a3-43e1-ba2e-6b8f4ff796c8'},
   

In [11]:
match_list = sample_data['data']['relationships']['matches']['data']
len(match_list), match_list[:5]

(317,
 [{'type': 'match', 'id': '762369d9-fd81-4b5a-9bd7-3aca6428a64b'},
  {'type': 'match', 'id': '1c7336ad-ab2e-43f5-8a15-4fdfffbee81c'},
  {'type': 'match', 'id': '023492b1-60b2-4c10-9a9c-3dde01168daa'},
  {'type': 'match', 'id': '4b997684-47d7-4c9a-9173-dc3fd26d75fd'},
  {'type': 'match', 'id': '2e6da0e6-effa-4331-864f-8ed4b8bad683'}])

Work on each match to get more information
- first save it into df
- then save the dataset into json file

In [12]:
features = ['date', 'match_id', 'duration', 'game_mode', 
            'num_players', 'num_teams', 'map']

In [13]:
agg_df = pd.DataFrame(columns = features)

In [14]:
agg_df

,date,match_id,duration,game_mode,num_players,num_teams,map


In [15]:
pubg = PUBG(api_key, shard = 'kakao')

In [16]:
error_list = []
for i, match in tqdm(enumerate(match_list)):
    try:
        match_id = match['id']
        current_match = pubg.match(match_id)
        telemetry = current_match.get_telemetry()
    
        date = telemetry.started()
        duration = telemetry.match_length()
        game_mode = current_match.game_mode
        num_players = telemetry.num_players()
        num_teams = telemetry.num_teams()
        map = telemetry.map_name()
    
        agg_df.loc[len(agg_df)] = [date, match_id, duration, 
                                   game_mode, num_players, num_teams, map]
    except:
        error_list.append(match_id)
        print(match['id'])
        continue

171it [19:54,  9.78s/it]

65343bd7-f547-4a0f-8d83-134316fd53ac


172it [19:59,  8.26s/it]

27449929-c664-4ab3-83b8-e68cffc982a6


317it [39:22,  7.45s/it]


In [17]:
agg_df.shape

(315, 7)

In [18]:
if not len(error_list) == None:
    pubg = PUBG(api_key, shard = 'kakao')
    for match in error_list:
        current_match = pubg.match(match)
        telemetry = current_match.get_telemetry()
        
        date = telemetry.started()
        duration = telemetry.match_length()
        game_mode = current_match.game_mode
        num_players = telemetry.num_players()
        num_teams = telemetry.num_teams()
        map = telemetry.map_name()
        
        agg_df.loc[len(agg_df)] = [date, match_id, duration, 
                                   game_mode, num_players, num_teams, map]
else:
    print("no need for work-up")

In [19]:
agg_df.shape, agg_df.head()

((317, 7),
                            date                              match_id  \
 0  2024-02-16T23:40:52.1395403Z  762369d9-fd81-4b5a-9bd7-3aca6428a64b   
 1  2024-02-16T23:31:14.0431751Z  1c7336ad-ab2e-43f5-8a15-4fdfffbee81c   
 2  2024-02-17T00:10:52.8557394Z  023492b1-60b2-4c10-9a9c-3dde01168daa   
 3  2024-02-17T00:17:07.2468332Z  4b997684-47d7-4c9a-9173-dc3fd26d75fd   
 4  2024-02-17T00:22:23.4293812Z  2e6da0e6-effa-4331-864f-8ed4b8bad683   
 
    duration game_mode  num_players  num_teams            map  
 0      1951     squad           92         18        Vikendi  
 1       369  clansolo            1          0    Camp Jackal  
 2      1363     squad          100         27         Sanhok  
 3      1439      solo            1          1  Tutorial_Main  
 4      1397     squad          130         27        Vikendi  )

#### save the aggregate data

In [20]:
json_data = agg_df.to_json(orient='records')
# len(json_data), json_data

In [21]:
json_file_name = f'data/{file_date}/{file_date}_agg_data.json'
json_file_name

'data/2024-02-17/2024-02-17_agg_data.json'

In [22]:
with open(json_file_name, 'w') as f:
    json.dump(json_data, f)

#### check existing file

In [18]:
import glob

In [20]:
check_file = glob.glob(f'data/{date}/*_agg_data.json')
check_file

['data/2024-02-11/2024-02-11_agg_data.json']

In [24]:
with open(check_file[0], 'r') as f:
    check_data = json.load(f)

In [25]:
check_data = json.loads(check_data)
check_data

[{'date': '2024-02-05T23:03:43.9531715Z',
  'match_id': '858cc73c-5926-497d-a975-a55b49975a12',
  'duration': 455,
  'game_mode': 'clansolo',
  'num_players': 1,
  'num_teams': 0,
  'map': 'Camp Jackal'},
 {'date': '2024-02-05T23:54:19.261471Z',
  'match_id': 'e2b8a957-c996-4d4b-af91-4d67d803a71c',
  'duration': 1549,
  'game_mode': 'solo',
  'num_players': 100,
  'num_teams': 100,
  'map': 'Erangel (Remastered)'},
 {'date': '2024-02-06T00:36:11.4290998Z',
  'match_id': '7a53a8aa-b9a1-4af4-82e7-2fa4f516d671',
  'duration': 1384,
  'game_mode': 'squad',
  'num_players': 100,
  'num_teams': 27,
  'map': 'Sanhok'},
 {'date': '2024-02-06T01:21:15.1780711Z',
  'match_id': 'ff9530d7-208b-46d5-a244-cc2f52bf258f',
  'duration': 1264,
  'game_mode': 'solo',
  'num_players': 100,
  'num_teams': 100,
  'map': 'Sanhok'},
 {'date': '2024-02-06T01:20:36.3554113Z',
  'match_id': '514db3d4-8547-414d-9f9e-04472b7a1b5f',
  'duration': 1334,
  'game_mode': 'duo',
  'num_players': 100,
  'num_teams': 51,


In [ ]:
# check_data = pd.DataFrame(check_data)
check_data.head()

In [ ]:
check_data.info()